# Search Contribution Overview

Use this notebook to track whether search is growing as a share of site traffic.

### Quick start (beginner-friendly)
1. Run the **Setup (run once)** cell.
2. In **Parameters**, choose a `DAYS_BACK` value (example: 28).
3. Run the remaining cells from top to bottom.

### Links
- GitHub repo: [github.com/aidanm-lla/lla-data](https://github.com/aidanm-lla/lla-data)
- Open this notebook in Colab: [Search Contribution Overview](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/01_search_contribution_overview.ipynb)

### Other notebooks
- [Top Pages Search Performance](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/02_top_pages_search_performance.ipynb)
- [Query Drivers by Page](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/03_query_drivers_by_page.ipynb)
- [Opportunity Watchlist](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/seo/04_opportunity_watchlist.ipynb)
- [Top Pages (Last 7 Days)](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/top_pages_last_7_days.ipynb)
- [Traffic Source Quality](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/traffic_sources.ipynb)
- [Time Patterns for Crisis-Related Pages](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/time_patterns.ipynb)
- [Crisis Support Funnel](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/crisis_funnel.ipynb)
- [Analysis Template](https://colab.research.google.com/github/aidanm-lla/lla-data/blob/main/notebooks/templates/analysis_template.ipynb)

Business questions:
- Are search clicks and organic sessions trending up?
- Is search share of total sessions improving?
- Are there recent drops that need investigation?

In [ ]:
#@title Setup (run once)
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    if not os.path.exists("lla-data"):
        !git clone -q https://github.com/aidoanto/lla-data.git
    repo = os.path.abspath("lla-data")
    if repo not in sys.path:
        sys.path.insert(0, repo)
    !pip install -q db-dtypes google-cloud-bigquery kaleido plotly
else:
    for p in ("..", "../.."): 
        ap = os.path.abspath(p)
        if ap not in sys.path:
            sys.path.insert(0, ap)

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()

In [ ]:
#@title Parameters
DAYS_BACK = 28 #@param {type:"integer"}

window = default_query_window(days_back=DAYS_BACK)

print(f"Project: {config.PROJECT_ID}")
print(f"Datasets: GA4={config.GA4_DATASET}, GSC={config.SEARCHCONSOLE_DATASET}")
print(f"Date range: {window.start_date} to {window.end_date}")

In [ ]:
from lla_data.bq import build_date_params

query = f"""
SELECT
  report_date,
  SUM(gsc_clicks) AS gsc_clicks,
  SUM(gsc_impressions) AS gsc_impressions,
  SUM(organic_sessions) AS organic_sessions,
  SUM(total_sessions) AS total_sessions,
  SAFE_DIVIDE(SUM(organic_sessions), NULLIF(SUM(total_sessions), 0)) AS search_share
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.seo_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
GROUP BY report_date
ORDER BY report_date
"""

df = run_query(client, query, params=build_date_params(window))
df.tail()

In [ ]:
fig_clicks = px.line(
    df,
    x="report_date",
    y=["gsc_clicks", "organic_sessions"],
    template="lifeline",
    title="Search Clicks vs Organic Sessions",
)
lifeline_theme.add_lifeline_logo(fig_clicks)
fig_clicks.show()

fig_share = px.line(
    df,
    x="report_date",
    y="search_share",
    template="lifeline",
    title="Search Share of Sessions",
)
fig_share.update_yaxes(tickformat=".0%")
lifeline_theme.add_lifeline_logo(fig_share)
fig_share.show()